### Import packages

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
from povertymapping.rollout_grids import get_region_filtered_bingtile_grid
from geowrangler.raster_zonal_stats import create_raster_zonal_stats
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import rasterio as rio
from rasterstats import zonal_stats

### Set global parameters


In [3]:
REGION = 'indonesia'
ADMIN_LVL = 'ADM2'

### Generate/Cache/Get per country grids

In [4]:
get_region_filtered_bingtile_grids?

Signature:
get_region_filtered_bingtile_grids(
    region: str,
    admin_lvl='ADM2',
    quadkey_lvl=14,
    use_cache=True,
    cache_dir='~/.cache/geowrangler',
    filter_population=True,
    assign_grid_admin_area=True,
    metric_crs='epsg:3857',
    extra_args={'nodata': nan},
) -> geopandas.geodataframe.GeoDataFrame
Docstring:
Get a geodataframe consisting of bing tile grids for a region/country at a quadkey level.
By default, the grids are filtered by population
Arguments:
   region: (required) the country/region for which grids will be created
   admin_lvl: (default: ADM2) the administrative level boundaries used for assigning the grids
   quadkey_lvl: (default: 14) the bingtile grid size zoom level
   use_cache: (default: True) whether to use a cached version or overwrite existing file
   cache_dir: (default: '~/.cache/geowrangler') directory where grids geojson will be created
   filter_population: (default: True) - whether to filter out grids with zero population counts
  

In [5]:
%%time
admin_grids_gdf = get_region_filtered_bingtile_grids(
    REGION, 
    admin_lvl=ADMIN_LVL, 
    filter_population=False
)

2023-03-06 15:30:03.342 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:56 - Loading cached grids file /home/jc_tm/.cache/geowrangler/quadkey_grids/indonesia_14_admin_grids.geojson


CPU times: user 20.2 s, sys: 450 ms, total: 20.7 s
Wall time: 20.7 s


### Explore per country populated grids

In [6]:
# admin_grids_gdf.explore()
# admin_grids_gdf.plot()


In [7]:
admin_grids_gdf.columns

Index(['quadkey', 'shapeName', 'shapeISO', 'shapeID', 'shapeGroup',
       'shapeType', 'geometry'],
      dtype='object')

In [8]:
admin_grids_gdf.head()

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,31000101131223,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.28369 -0.52734, 98.28369 -0.50536..."
1,31000101133001,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.28369 -0.54931, 98.28369 -0.52734..."
2,31000101131232,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.30566 -0.52734, 98.30566 -0.50536..."
3,31000101133010,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.30566 -0.54931, 98.30566 -0.52734..."
4,31000101131231,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.32764 -0.50536, 98.32764 -0.48339..."


In [9]:
admin_grids_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 340122 entries, 0 to 340121
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   quadkey     340122 non-null  object  
 1   shapeName   340122 non-null  object  
 2   shapeISO    340122 non-null  object  
 3   shapeID     340122 non-null  object  
 4   shapeGroup  340122 non-null  object  
 5   shapeType   340122 non-null  object  
 6   geometry    340122 non-null  geometry
dtypes: geometry(1), object(6)
memory usage: 18.2+ MB


## Test Optimizations for Indonesia Filtering

In [10]:
admin_grids_gdf.head()

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,31000101131223,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.28369 -0.52734, 98.28369 -0.50536..."
1,31000101133001,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.28369 -0.54931, 98.28369 -0.52734..."
2,31000101131232,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.30566 -0.52734, 98.30566 -0.50536..."
3,31000101133010,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.30566 -0.54931, 98.30566 -0.52734..."
4,31000101131231,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.32764 -0.50536, 98.32764 -0.48339..."


In [11]:
from povertymapping.hdx import get_hdx_file, get_unzipped_hdxfile, get_hrsl_dataset
hdx_pop_file = get_hdx_file(REGION)
hdx_pop_file

Path('/home/jc_tm/.cache/geowrangler/hdx/idn_general_2020.tif')

In [12]:
(hdx_pop_file.stat().st_size) / 1e9

87.714317324

In [13]:
with rio.open(hdx_pop_file) as dst:
    print(dst.profile)
    print(dst.crs)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': nan, 'width': 169200, 'height': 64800, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777780012, 0.0, 94.99986111133225,
       0.0, -0.0002777777777780012, 7.000138888837142), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}
EPSG:4326


In [14]:
admin_grids_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [15]:
hdx_pop_file.is_file()

True

## Filter out Danau grids
For Indonesia, we filter out the grids found to be in the admin bound `Danau` (English translation: lake) as it actually corresponds to grids over landlocked lakes. Running rasterstats on these grids causes it to crash as it loads a very big raster. We also assume that these areas are unpopulated as they cover bodies of water. 

In [16]:
danau_gdf = admin_grids_gdf[admin_grids_gdf['shapeName'] == 'Danau']
danau_gdf.explore()

In [17]:
admin_grids_gdf_filtered = admin_grids_gdf[admin_grids_gdf['shapeName'] != 'Danau']

In [18]:
admin_grids_gdf_filtered.head()

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,31000101131223,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.28369 -0.52734, 98.28369 -0.50536..."
1,31000101133001,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.28369 -0.54931, 98.28369 -0.52734..."
2,31000101131232,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.30566 -0.52734, 98.30566 -0.50536..."
3,31000101133010,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.30566 -0.54931, 98.30566 -0.52734..."
4,31000101131231,Nias Selatan,None,IDN-ADM2-3_0_0-B371,IDN,ADM2,"POLYGON ((98.32764 -0.50536, 98.32764 -0.48339..."


In [19]:
admin_grids_gdf_filtered.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 340075 entries, 0 to 340121
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   quadkey     340075 non-null  object  
 1   shapeName   340075 non-null  object  
 2   shapeISO    340075 non-null  object  
 3   shapeID     340075 non-null  object  
 4   shapeGroup  340075 non-null  object  
 5   shapeType   340075 non-null  object  
 6   geometry    340075 non-null  geometry
dtypes: geometry(1), object(6)
memory usage: 20.8+ MB


## Process each grid based on `shapeName`

In [20]:
area_list = admin_grids_gdf_filtered['shapeID'].unique()
len(area_list)

518

In [21]:
# area_pop_count_df_list = []
# with rio.open(hdx_pop_file) as dst:
#     for area in tqdm(area_list):
#         area_grids_gdf = (
#             admin_grids_gdf[admin_grids_gdf["shapeID"] == area]
#             .reset_index(drop=True)
#             .copy()
#         )

#         # Get geometries and bounds for the specified area
#         area_bounds = area_grids_gdf.total_bounds
#         left, bottom, right, top = area_bounds

#         # Get the data (np.array, affine transform) for the
#         # window specified by the area bounds
#         window = rio.windows.from_bounds(left, bottom, right, top, dst.transform)
#         window_transform = dst.window_transform(window)
#         area_population = dst.read(1, window=window)
#         area_population = np.float32(area_population)

#         # Sum up the population counts for each grid in area_grids_gdf
#         # and store as a column 'pop_count'
#         area_grids_gdf["pop_count"] = pd.DataFrame(
#             zonal_stats(
#                 vectors=area_grids_gdf,
#                 raster=area_population,
#                 affine=window_transform,
#                 stats="sum",
#                 all_touched=True,
#                 nodata=np.nan,
#             )
#         )["sum"].fillna(0, inplace=True)

#         # Get dataframe containing quadkey and pop_count
#         area_pop_count_df = area_grids_gdf[["quadkey", "pop_count"]]
#         area_pop_count_df_list.append(area_pop_count_df)

# pop_count = pd.concat(area_pop_count_df_list)


In [22]:
1+1

2

In [23]:
CHUNK_SIZE = 1000
area_pop_count_df_list = []
with rio.open(hdx_pop_file) as dst:
    for area in tqdm(area_list):
        area_grids_gdf = (
            admin_grids_gdf[admin_grids_gdf["shapeID"] == area]
            .reset_index(drop=True)
            .copy()
        )

        # Loop through area in chunks
        for chunk_num in range(len(area_grids_gdf) // CHUNK_SIZE + 1):
            
            start_index = chunk_num*CHUNK_SIZE
            end_index = min(chunk_num*CHUNK_SIZE + CHUNK_SIZE, len(area_grids_gdf))
            chunk_area_grids_gdf = area_grids_gdf[start_index:end_index].copy()
            
            # Get geometries and bounds for the specified chunk
            chunk_bounds = area_grids_gdf.total_bounds
            left, bottom, right, top = chunk_bounds

            # Get the data (np.array, affine transform) for the
            # window specified by the chunk bounds
            window = rio.windows.from_bounds(left, bottom, right, top, dst.transform)
            window_transform = dst.window_transform(window)
            chunk_population = dst.read(1, window=window)
            chunk_population = np.float32(chunk_population)

            # Sum up the population counts for each grid in area_grids_gdf
            # and store as a column 'pop_count'
            chunk_area_grids_gdf["pop_count"] = pd.DataFrame(
                zonal_stats(
                    vectors=chunk_area_grids_gdf,
                    raster=chunk_population,
                    affine=window_transform,
                    stats="sum",
                    all_touched=True,
                    nodata=np.nan,
                )
            )["sum"]

            chunk_area_pop_count_df = chunk_area_grids_gdf[["quadkey", "pop_count"]]
            area_pop_count_df_list.append(chunk_area_pop_count_df)

# Combine populuation count dataframes
pop_count = pd.concat(area_pop_count_df_list)


  9%|▉         | 47/518 [03:39<12:21,  1.58s/it]  

In [ ]:
pop_count = pd.concat(area_pop_count_df_list)
pop_count

In [ ]:
len(area_grids_gdf)